### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [78]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

### read in policy data 

#### china subnational data

In [7]:
url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/China/OxCGRT_CHN_latest%20LEGACY.csv"

In [8]:
china_df = pd.read_csv(url)

In [9]:
china_df.columns = china_df.columns.str.lower().str.replace(" ", "_")

In [10]:
china_df["date"] = pd.to_datetime(china_df["date"], format="%Y%m%d")

In [11]:
china_df.sort_values("date", ascending=False)

#### international indices

In [72]:
url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_nat_differentiated_withnotes_2022.csv"

In [73]:
df = pd.read_csv(url)

In [74]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [75]:
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

In [76]:
df.sort_values("date", ascending=False)

,countryname,countrycode,regionname,regioncode,jurisdiction,date,c1e_school_closing,c1e_flag,c1nv_school_closing,c1nv_flag,c1v_school_closing,c1v_flag,c1m_school_closing,c1m_flag,c1_notes,c2e_workplace_closing,c2e_flag,c2nv_workplace_closing,c2nv_flag,c2v_workplace_closing,c2v_flag,c2m_workplace_closing,c2m_flag,c2_notes,c3e_cancel_public_events,c3e_flag,c3nv_cancel_public_events,c3nv_flag,c3v_cancel_public_events,c3v_flag,c3m_cancel_public_events,c3m_flag,c3_notes,c4e_restrictions_on_gatherings,c4e_flag,c4nv_restrictions_on_gatherings,c4nv_flag,c4v_restrictions_on_gatherings,c4v_flag,c4m_restrictions_on_gatherings,c4m_flag,c4_notes,c5e_close_public_transport,c5e_flag,c5nv_close_public_transport,c5nv_flag,c5v_close_public_transport,c5v_flag,c5m_close_public_transport,c5m_flag,c5_notes,c6e_stay_at_home_requirements,c6e_flag,c6nv_stay_at_home_requirements,c6nv_flag,c6v_stay_at_home_requirements,c6v_flag,c6m_stay_at_home_requirements,c6m_flag,c6_notes,c7e_restrictions_on_internal_movement,c7e_flag,c7nv_restrictions_on_internal_movement,c7nv_flag,c7v_restrictions_on_internal_movement,c7v_flag,c7m_restrictions_on_internal_movement,c7m_flag,c7_notes,c8e_international_travel_controls,c8nv_international_travel_controls,c8v_international_travel_controls,c8ev_international_travel_controls,c8_notes,e1e_income_support,e1e_flag,e1_notes,e2e_debt/contract_relief,e2_notes,e3e_fiscal_measures,e3_notes,e4e_international_support,e4_notes,h1e_public_information_campaigns,h1e_flag,h1_notes,h2e_testing_policy,h2_notes,h3e_contact_tracing,h3_notes,h4e_emergency_investment_in_healthcare,h4_notes,h5e_investment_in_vaccines,h5_notes,h6e_facial_coverings,h6e_flag,h6nv_facial_coverings,h6nv_flag,h6v_facial_coverings,h6v_flag,h6m_facial_coverings,h6m_flag,h6_notes,h7e_vaccination_policy,h7e_flag,h7_notes,h8e_protection_of_elderly_people,h8e_flag,h8nv_protection_of_elderly_people,h8nv_flag,h8v_protection_of_elderly_people,h8v_flag,h8m_protection_of_elderly_people,h8m_flag,h8_notes,m1e_wildcard,m1_notes,v1_vaccine_prioritisation_(summary),v1_notes,v2a_vaccine_availability_(summary),v2_notes,v2b_vaccine_age_eligibility/availability_age_floor_(general_population_summary),v2c_vaccine_age_eligibility/availability_age_floor_(at_risk_summary),v2d_medically/_clinically_vulnerable_(non-elderly),v2e_education,v2f_frontline_workers__(non_healthcare),v2g_frontline_workers__(healthcare),v3_vaccine_financial_support_(summary),v3_notes,v4_mandatory_vaccination_(summary),v4_notes,confirmedcases,confirmeddeaths,populationvaccinated,stringencyindex_nonvaccinated,stringencyindex_nonvaccinated_fordisplay,stringencyindex_vaccinated,stringencyindex_vaccinated_fordisplay,stringencyindex_simpleaverage,stringencyindex_simpleaverage_fordisplay,stringencyindex_weightedaverage,stringencyindex_weightedaverage_fordisplay,governmentresponseindex_nonvaccinated,governmentresponseindex_nonvaccinated_fordisplay,governmentresponseindex_vaccinated,governmentresponseindex_vaccinated_fordisplay,governmentresponseindex_simpleaverage,governmentresponseindex_simpleaverage_fordisplay,governmentresponseindex_weightedaverage,governmentresponseindex_weightedaverage_fordisplay,containmenthealthindex_nonvaccinated,containmenthealthindex_nonvaccinated_fordisplay,containmenthealthindex_vaccinated,containmenthealthindex_vaccinated_fordisplay,containmenthealthindex_simpleaverage,containmenthealthindex_simpleaverage_fordisplay,containmenthealthindex_weightedaverage,containmenthealthindex_weightedaverage_fordisplay,economicsupportindex,economicsupportindex_fordisplay
47123,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

#### just mapping china's stringency index over time (not subnational)

In [12]:
china_slim = china_df[china_df["jurisdiction"] == "NAT_TOTAL"][
    ["date", "stringencyindex"]
]

In [13]:
alt.Chart(china_slim).mark_line().encode(x="date", y="stringencyindex")

alt.Chart(...)

In [35]:
#### make a datawrapper line chart for China's stringency index over time
#dw.create_chart("China policies over time", chart_type="d3-lines", data=china_slim)
#chart_id=D2Swg

#### get us data for comparison

In [81]:
df.head()

,countryname,countrycode,regionname,regioncode,jurisdiction,date,c1e_school_closing,c1e_flag,c1nv_school_closing,c1nv_flag,c1v_school_closing,c1v_flag,c1m_school_closing,c1m_flag,c1_notes,c2e_workplace_closing,c2e_flag,c2nv_workplace_closing,c2nv_flag,c2v_workplace_closing,c2v_flag,c2m_workplace_closing,c2m_flag,c2_notes,c3e_cancel_public_events,c3e_flag,c3nv_cancel_public_events,c3nv_flag,c3v_cancel_public_events,c3v_flag,c3m_cancel_public_events,c3m_flag,c3_notes,c4e_restrictions_on_gatherings,c4e_flag,c4nv_restrictions_on_gatherings,c4nv_flag,c4v_restrictions_on_gatherings,c4v_flag,c4m_restrictions_on_gatherings,c4m_flag,c4_notes,c5e_close_public_transport,c5e_flag,c5nv_close_public_transport,c5nv_flag,c5v_close_public_transport,c5v_flag,c5m_close_public_transport,c5m_flag,c5_notes,c6e_stay_at_home_requirements,c6e_flag,c6nv_stay_at_home_requirements,c6nv_flag,c6v_stay_at_home_requirements,c6v_flag,c6m_stay_at_home_requirements,c6m_flag,c6_notes,c7e_restrictions_on_internal_movement,c7e_flag,c7nv_restrictions_on_internal_movement,c7nv_flag,c7v_restrictions_on_internal_movement,c7v_flag,c7m_restrictions_on_internal_movement,c7m_flag,c7_notes,c8e_international_travel_controls,c8nv_international_travel_controls,c8v_international_travel_controls,c8ev_international_travel_controls,c8_notes,e1e_income_support,e1e_flag,e1_notes,e2e_debt/contract_relief,e2_notes,e3e_fiscal_measures,e3_notes,e4e_international_support,e4_notes,h1e_public_information_campaigns,h1e_flag,h1_notes,h2e_testing_policy,h2_notes,h3e_contact_tracing,h3_notes,h4e_emergency_investment_in_healthcare,h4_notes,h5e_investment_in_vaccines,h5_notes,h6e_facial_coverings,h6e_flag,h6nv_facial_coverings,h6nv_flag,h6v_facial_coverings,h6v_flag,h6m_facial_coverings,h6m_flag,h6_notes,h7e_vaccination_policy,h7e_flag,h7_notes,h8e_protection_of_elderly_people,h8e_flag,h8nv_protection_of_elderly_people,h8nv_flag,h8v_protection_of_elderly_people,h8v_flag,h8m_protection_of_elderly_people,h8m_flag,h8_notes,m1e_wildcard,m1_notes,v1_vaccine_prioritisation_(summary),v1_notes,v2a_vaccine_availability_(summary),v2_notes,v2b_vaccine_age_eligibility/availability_age_floor_(general_population_summary),v2c_vaccine_age_eligibility/availability_age_floor_(at_risk_summary),v2d_medically/_clinically_vulnerable_(non-elderly),v2e_education,v2f_frontline_workers__(non_healthcare),v2g_frontline_workers__(healthcare),v3_vaccine_financial_support_(summary),v3_notes,v4_mandatory_vaccination_(summary),v4_notes,confirmedcases,confirmeddeaths,populationvaccinated,stringencyindex_nonvaccinated,stringencyindex_nonvaccinated_fordisplay,stringencyindex_vaccinated,stringencyindex_vaccinated_fordisplay,stringencyindex_simpleaverage,stringencyindex_simpleaverage_fordisplay,stringencyindex_weightedaverage,stringencyindex_weightedaverage_fordisplay,governmentresponseindex_nonvaccinated,governmentresponseindex_nonvaccinated_fordisplay,governmentresponseindex_vaccinated,governmentresponseindex_vaccinated_fordisplay,governmentresponseindex_simpleaverage,governmentresponseindex_simpleaverage_fordisplay,governmentresponseindex_weightedaverage,governmentresponseindex_weightedaverage_fordisplay,containmenthealthindex_nonvaccinated,containmenthealthindex_nonvaccinated_fordisplay,containmenthealthindex_vaccinated,containmenthealthindex_vaccinated_fordisplay,containmenthealthindex_simpleaverage,containmenthealthindex_simpleaverage_fordisplay,containmenthealthindex_weightedaverage,containmenthealthindex_weightedaverage_fordisplay,economicsupportindex,economicsupportindex_fordisplay
0,Aruba,ABW,NaN,NaN,NAT_TOTAL,2022-01-01,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,4.0,1.0,NaN,NaN,NaN,NaN,4.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,0.0,NaN,2.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,5.0,1.0,NaN,0.0,NaN,

In [82]:
us_china = ["United States", "China"]

us_china_wide = df[df["countryname"].isin(us_china)][
    ["date", "countryname", "stringencyindex_weightedaverage_fordisplay"]
]

In [84]:
us_china_wide = (
    us_china_wide.pivot(
        index="date",
        columns=["countryname"],
        values="stringencyindex_weightedaverage_fordisplay",
    )
    .reset_index()
    .copy()
)

In [86]:
# dw.create_chart(
#    "Severity of Covid-19 policies over time",
#    chart_type="d3-lines",
#    data=us_china_wide[us_china_wide["date"] >= "2022-01-01"],
# )
chart_id = "tV5c3"
dw.add_data(
    chart_id=chart_id, data=us_china_wide[us_china_wide["date"] >= "2021-09-09"]
)

<Response [204]>

#### china provinces most current stringency indices

In [15]:
## here are the most recent dates with available data for each region
china_max_dates = (
    china_df[(~china_df["stringencyindex"].isna())]
    .groupby("regionname", as_index=False)
    .agg({"date": "max"})
    .sort_values("date", ascending=True)
)

In [37]:
provincial_stringency = pd.merge(
    china_df[["regionname", "date", "stringencyindex"]],
    china_max_dates,
    on=["regionname", "date"],
)

In [54]:
# add data to map of china
dw.add_data(chart_id="5y7XX", data=provincial_stringency)

<Response [204]>

### group  by date and count the number of '3' (strict) c6_stay_at_home_requirements over time (how many provinces have the strictest stay at home requirement in place on a given day)

In [40]:
stayathome_overtime = china_df[
    (~china_df["c6_stay_at_home_requirements"].isna())
    & (china_df["jurisdiction"] != "NAT_TOTAL")
][["regionname", "date", "c6_stay_at_home_requirements"]]

In [42]:
stayathome_counts = (
    stayathome_overtime.groupby(["date", "c6_stay_at_home_requirements"])
    .size()
    .reset_index(name="count")
)

In [19]:
# plotting just the number of regions with the strictest lockdowns in place
alt.Chart(
    stayathome_counts[stayathome_counts["c6_stay_at_home_requirements"] == 3]
).mark_bar().encode(x="date", y="count", color="c6_stay_at_home_requirements:N")

In [43]:
#dw.create_chart(
#    "Stay at home measures over time",
#    chart_type="d3-area",
#    data=stayathome_counts[stayathome_counts["c6_stay_at_home_requirements"] == 3],
#)
# chart_id= '5xbhj'

In [44]:
# see which provinces currently have strict stay at home in place
stayathome_overtime[
    (stayathome_overtime["date"] == stayathome_overtime["date"].max())
    & (stayathome_overtime["c6_stay_at_home_requirements"] == 3)
]

,regionname,date,c6_stay_at_home_requirements
980,Anhui,2022-09-07,3.0
5895,Gansu,2022-09-07,3.0
21623,Sichuan,2022-09-07,3.0


In [50]:
current_stayathome = china_df[china_df["date"] == "2022-09-07"][
    ["regionname", "date", "c6_stay_at_home_requirements"]
]

In [55]:
dw.add_data(
    chart_id="Vpinw",
    data=current_stayathome,
)

<Response [204]>